In [1]:
import stim
from stimrgs_v1.utils import *
from stimrgs_v1.stabilizer_operations import *
from tqdm import tqdm

In [4]:
# Configuration parameters
num_trial = 5
num_nodes = 20
num_bell_between_row = [i+2 for i in range(num_nodes//4 -1)]

stabilizers_results = []
stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

for num_bell in num_bell_between_row:

    # Reset max_found_bells every time we change num_bell_between_row
    max_found_bells = 0

    for trial in tqdm(range(num_trial)):        

        # for num_measurements in tqdm(range(5, num_nodes - 3)):
        for num_measurements in [4,5,6,7,8]:
        # num_measurements = 16                         
            combinations = fast_generate_combinations_with_adjustable_replacement(
                nodes=[i for i in range(num_nodes)], 
                max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
                max_self_combination=1
            )                    

            for c in combinations: 
                # Restart parameters before operate new measurement set
                found: bool = False
                bell_found_list = []

                q, circuit = generate_rgs_random(num_nodes=num_nodes, num_bell_between_row=num_bell)
                s = stim.TableauSimulator()
                s.do_circuit(stim.Circuit(circuit))

                # Operate measurement
                for index in c:
                    # Update s, q
                    s, q = measure_z_with_correction(index=index, s=s, queries=q)

                # Check if result match one of all possible results
                for result in stabilizers_results:
                    if is_subset(result, q):
                        # print(f"Found solution: {result}")   
                        bell_found_list.append(result)                     
                        found = True                       
                
                # Update when found new high
                if len(bell_found_list) >= max_found_bells:
                    max_found_bells = len(bell_found_list)

    print(f'Max found bells {num_bell}: {max_found_bells}')

100%|██████████| 5/5 [1:06:54<00:00, 802.98s/it]


Max found bells 2: 3


100%|██████████| 5/5 [1:10:21<00:00, 844.38s/it]


Max found bells 3: 2


100%|██████████| 5/5 [1:03:55<00:00, 767.01s/it]


Max found bells 4: 0


 20%|██        | 1/5 [20:33<1:22:13, 1233.39s/it]


KeyboardInterrupt: 

In [ ]:
# Configuration parameters
num_trial = 5
num_nodes = 20
num_bell_between_row = [i+2 for i in range(num_nodes//4 -1)]

stabilizers_results = []
stabilizers_results = find_stabilizers_result(num_nodes=num_nodes)

for num_bell in num_bell_between_row:

    # Reset max_found_bells every time we change num_bell_between_row
    max_found_bells = 0

    for trial in tqdm(range(num_trial)):        

        # for num_measurements in tqdm(range(5, num_nodes - 3)):
        for num_measurements in [4,5,6,7,8]:
        # num_measurements = 16                         
            combinations = fast_generate_combinations_with_adjustable_replacement(
                nodes=[i for i in range(num_nodes)], 
                max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
                max_self_combination=1
            )                    

            for c in combinations: 
                # Restart parameters before operate new measurement set
                found: bool = False
                bell_found_list = []

                q, circuit = generate_rgs_random(num_nodes=num_nodes, num_bell_between_row=num_bell)
                s = stim.TableauSimulator()
                s.do_circuit(stim.Circuit(circuit))

                # Operate measurement
                for index in c:
                    # Update s, q
                    # s, q = measure_z_with_correction(index=index, s=s, queries=q)
                    s.postselect_z(index, desired_value=False)

                # Check if result match one of all possible results
                for result in stabilizers_results:
                    if is_subset(result, q):
                        # print(f"Found solution: {result}")   
                        bell_found_list.append(result)                     
                        found = True                       
                
                # Update when found new high
                if len(bell_found_list) >= max_found_bells:
                    max_found_bells = len(bell_found_list)

    print(f'Max found bells {num_bell}: {max_found_bells}')

In [36]:
first_station = '0'
middle_edge = '3 6'
# full mid-edge = ['3 6', '4 7', '5, 8']
end_station = '9'

bell_circuit = stim.Circuit(f'''
    H 0 1 2 3 4 5 6 7 8 9 10 11
    CZ {first_station} 3 {middle_edge} 6 {end_station}
''')

q, circuit = generate_rgs_random(num_nodes=12, num_bell_between_row=3)
s = stim.TableauSimulator()
# s.do_circuit(stim.Circuit(circuit))
s.do_circuit(bell_circuit)

for i in [1,2,4,5,7,8,10,11]:
    s.postselect_z(i, desired_value=False)

s.canonical_stabilizers()

[stim.PauliString("+X_____X__Z__"),
 stim.PauliString("+Z__X_____X__"),
 stim.PauliString("+_Z__________"),
 stim.PauliString("+__Z_________"),
 stim.PauliString("+___Z__X__Z__"),
 stim.PauliString("+____Z_______"),
 stim.PauliString("+_____Z______"),
 stim.PauliString("+______Z__X__"),
 stim.PauliString("+_______Z____"),
 stim.PauliString("+________Z___"),
 stim.PauliString("+__________Z_"),
 stim.PauliString("+___________Z")]

In [ ]:
def all_bell_result(firsts, middles, ends):
    all_bell_result = []    
    for first in firsts:
        for middle in middles:
            for end in ends:

                elements = middle.split()
                duplicated_elements = [element for element in elements for _ in range(2)]
                result = ' '.join(duplicated_elements)

                numbers = [first + ' ' + middle + ' ' +  end]
                unique_numbers = sorted(set(numbers), key=numbers.index)
                h_line = "H " + " ".join(map(str, sorted(unique_numbers)))

                circuit = f'''{h_line}\nCZ {first} {result} {end}'''

                s = stim.TableauSimulator()
                s.do_circuit(stim.Circuit(circuit))

                # all_bell_result.append(stim.Circuit(circuit))
                all_bell_result.append(s)

    return all_bell_result

first = ['0', '1', '2']
middle = ['3 6', '4 7', '5 8']
end = ['9', '10', '11']

result = all_bell_result(first, middle, end)

In [79]:
result[0].canonical_stabilizers()

[stim.PauliString("+X_____X__Z"),
 stim.PauliString("+Z__X_____X"),
 stim.PauliString("+_Z________"),
 stim.PauliString("+__Z_______"),
 stim.PauliString("+___Z__X__Z"),
 stim.PauliString("+____Z_____"),
 stim.PauliString("+_____Z____"),
 stim.PauliString("+______Z__X"),
 stim.PauliString("+_______Z__"),
 stim.PauliString("+________Z_")]

In [80]:
q = [
    'X___ZZ______', # node 0
    '_X_Z_Z______', # node 1
    '__XZZ_______', # node 2
    '_ZZX__Z_____', # node 3
    'Z_Z_X__Z____', # node 4
    'ZZ___X__Z___', # node 5
    '___Z__X___ZZ', # node 6
    '____Z__X_Z_Z', # node 7
    '_____Z__XZZ_', # node 8
    '_______ZZX__', # node 9
    '______Z_Z_X_', # node 10
    '______ZZ___X'  # node 11
]

s = stim.TableauSimulator()
s.do_circuit(stim.Circuit('''
    H 0 1 2 3 4 5 6 7 8 9 10 11
    CZ 0 4 0 5 1 3 1 5 2 3 2 4 6 10 6 11 7 9 7 11 8 9 8 10 3 6 4 7 5 8    
'''))

for i in [1,4,7,10]:
    s.postselect_z(i, desired_value=False)

s.canonical_stabilizers()

[stim.PauliString("+X_______XZ__"),
 stim.PauliString("+Z____X___X__"),
 stim.PauliString("+_Z__________"),
 stim.PauliString("+__X___X____Z"),
 stim.PauliString("+__ZX_______X"),
 stim.PauliString("+___Z__X____Z"),
 stim.PauliString("+____Z_______"),
 stim.PauliString("+_____Z__XZ__"),
 stim.PauliString("+______Z____X"),
 stim.PauliString("+_______Z____"),
 stim.PauliString("+________ZX__"),
 stim.PauliString("+__________Z_")]

In [59]:
%%timeit

find_stabilizers_result(num_nodes=12)

140 µs ± 3.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [85]:
circuit = stim.Circuit('''
    H 0 1 2 3 
    CZ 0 1 1 2 2 3 3 4
''')
s = stim.TableauSimulator()
s.do_circuit(circuit)

s.postselect_x(1, desired_value=False)
s.postselect_x(2, desired_value=False)

s.current_inverse_tableau()

queries = [
    'X___Z',
    'Z___X'
]

for q in queries:
    print(s.peek_observable_expectation(stim.PauliString(q)))

0
0
